In [ ]:
# todo NOTE: this script does not reproduce original result since we removed device IDs 
# todo NOTE: also this script will encounter runtime/space error since we used device IDs as a key while merge dataframes 

import pandas as pd
import numpy as np

file_cleaned_flow = '../../../Endpoint Mapping Data/Cleaned Flow/cleaned_flow_stat.csv'
cleaned_flow = pd.read_csv(file_cleaned_flow)

cleaned_flow['super_vendor'] = cleaned_flow.apply(
    lambda row: row.vendor_name.lower() if row.vendor_name==row.vendor_name
    else row.gpt_clean_vendor,
    axis=1)


cleaned_flow['generic_category'] = cleaned_flow.apply(
    lambda row: row.man_generic_category if row.man_generic_category==row.man_generic_category
    else row.gpt_generic_category,
    axis=1)


# read all party mapping files
file_all_party_mapping = '../../../Endpoint Mapping Data/Domain Data/all_party_mapping.csv'
all_party_mapping = pd.read_csv(file_all_party_mapping)
# drop extra columns 
all_party_mapping = all_party_mapping[['super_vendor', 'domain', 'party_labels']].drop_duplicates()

# marge with clean flow
clean_flow_party_label = pd.merge(cleaned_flow,
                                  all_party_mapping,
                                  on=['super_vendor', 'domain'],
                                  how='left'
                                  )


# read user information file 
file_user_device_timezone = '../../../Inspector Dataset/New data/user_device_timezone.csv'
user_device_timezone = pd.read_csv(file_user_device_timezone)
user_device_timezone = user_device_timezone[['device_id', 'user_key', 'user_country', 'timezone']]

# merge with timezone file 
clean_flow_party_label = pd.merge(clean_flow_party_label,
                                  user_device_timezone,
                                  on=['device_id'],
                                  how='left'
                                  )

unique_categories = ['Media/TV',
                     'Home Automation',
                     'Voice Assistant',
                     'Surveillance',
                     'Game Console',
                     'Work Appliance',
                     'Home Appliance ',
                     'Generic IoT',]
print(len(clean_flow_party_label['device_id'].unique()))

clean_flow_party_label['average_out_byte_per_sec'] = clean_flow_party_label['total_out_byte']/clean_flow_party_label['flow_duration']
threshold = 1e6
# Replace values greater than the threshold with NaN
clean_flow_party_label['average_out_byte_per_sec'] = np.where(clean_flow_party_label['average_out_byte_per_sec'] > threshold,
                                                              np.nan, clean_flow_party_label['average_out_byte_per_sec'])

## RQ 4 (Longitudinal Analysis)

In [ ]:
import json
file_location = '../../../Statistical Data/RQ3/country-list.json'

with open(file_location, encoding='utf-8') as f:
    country_2_continent = json.load(f)

# Create a list of tuples containing key-value pairs
country_2_continent = [(key, value) for key, value in country_2_continent.items()]
country_2_continent = pd.DataFrame(country_2_continent, columns=['remote_ip_country', 'domain_loc'])

# find continent to remote ip country 
clean_flow_party_label  = pd.merge(clean_flow_party_label, country_2_continent, on=['remote_ip_country'], how='left')

In [ ]:
# function to find the timezone of the users
def timezone_continent(_timezone):
    try:
        _timezone = float(_timezone)
        if (_timezone >= -11) and (_timezone < -1):
            return 'AMERICAS'
        if (_timezone >= 0) and (_timezone <= 3):
            return 'EA'
        if _timezone >= 4:
            return 'APA'
    except:
        pass
    return 'UN'


timezone = clean_flow_party_label[['device_id', 'timezone']].drop_duplicates()
timezone['user_loc'] =  timezone.apply(lambda row: timezone_continent(row.timezone), axis=1)

# find user location based on timezone
clean_flow_party_label = pd.merge(clean_flow_party_label, timezone, on=['device_id', 'timezone'], how='left')
print(len(clean_flow_party_label['device_id'].unique()))

In [ ]:
from datetime import datetime

ts = clean_flow_party_label['min_ts'].min()
dt = datetime.fromtimestamp(ts)
print("The date and time is:", dt)

ts = clean_flow_party_label['min_ts'].max()
dt = datetime.fromtimestamp(ts)
print("The date and time is:", dt)

threshold = datetime.strptime("2020-12-31 23:59:59", '%Y-%m-%d %H:%M:%S').timestamp()
print(threshold)

In [ ]:
months_timestamp = ["2019-03-31 23:59:59", "2019-04-30 23:59:59", "2019-05-31 23:59:59", "2019-06-30 23:59:59", "2019-07-31 23:59:59", "2019-08-31 23:59:59",
                    "2019-09-30 23:59:59", "2019-10-31 23:59:59", "2019-11-30 23:59:59", "2019-12-31 23:59:59", "2020-01-31 23:59:59", "2020-02-29 23:59:59",
                    "2020-03-31 23:59:59", "2020-04-30 23:59:59", "2020-05-31 23:59:59", "2020-06-30 23:59:59", "2020-07-31 23:59:59", "2020-08-31 23:59:59",
                    "2020-09-30 23:59:59", "2020-10-31 23:59:59", "2020-11-30 23:59:59", "2020-12-31 23:59:59", "2021-01-31 23:59:59", "2021-02-28 23:59:59",
                    "2021-03-31 23:59:59", "2021-04-30 23:59:59", "2021-05-31 23:59:59", "2021-06-30 23:59:59", "2021-07-31 23:59:59", "2021-08-31 23:59:59",
                    "2021-09-30 23:59:59", "2021-10-31 23:59:59", "2021-11-30 23:59:59", "2021-12-31 23:59:59", "2022-01-31 23:59:59", "2022-02-28 23:59:59",
                    "2022-03-31 23:59:59", "2022-04-30 23:59:59", "2022-05-31 23:59:59", "2022-06-30 23:59:59", "2022-07-31 23:59:59"]


month_map = {"2019-03-31 23:59:59" : "Mar'19",
             "2019-04-30 23:59:59" : "Apr'19",
             "2019-05-31 23:59:59" : "May'19",
             "2019-06-30 23:59:59" : "Jun'19",
             "2019-07-31 23:59:59" : "Jul'19",
             "2019-08-31 23:59:59" : "Aug'19",
             "2019-09-30 23:59:59" : "Sep'19",
             "2019-10-31 23:59:59" : "Oct'19",
             "2019-11-30 23:59:59" : "Nov'19",
             "2019-12-31 23:59:59" : "Dec'19",
             "2020-01-31 23:59:59" : "Jan'20",
             "2020-02-29 23:59:59" : "Feb'20",
             "2020-03-31 23:59:59" : "Mar'20",
             "2020-04-30 23:59:59" : "Apr'20",
             "2020-05-31 23:59:59" : "May'20",
             "2020-06-30 23:59:59" : "Jun'20",
             "2020-07-31 23:59:59" : "Jul'20",
             "2020-08-31 23:59:59" : "Aug'20",
             "2020-09-30 23:59:59" : "Sep'20",
             "2020-10-31 23:59:59" : "Oct'20",
             "2020-11-30 23:59:59" : "Nov'20",
             "2020-12-31 23:59:59" : "Dec'20",
             "2021-01-31 23:59:59" : "Jan'21",
             "2021-02-28 23:59:59" : "Feb'21",
             "2021-03-31 23:59:59" : "Mar'21",
             "2021-04-30 23:59:59" : "Apr'21",
             "2021-05-31 23:59:59" : "May'21",
             "2021-06-30 23:59:59" : "Jun'21",
             "2021-07-31 23:59:59" : "Jul'21",
             "2021-08-31 23:59:59" : "Aug'21",
             "2021-09-30 23:59:59" : "Sep'21",
             "2021-10-31 23:59:59" : "Oct'21",
             "2021-11-30 23:59:59" : "Nov'21",
             "2021-12-31 23:59:59" : "Dec'21",
             "2022-01-31 23:59:59" : "Jan'22",
             "2022-02-28 23:59:59" : "Feb'22",
             "2022-03-31 23:59:59" : "Mar'22",
             "2022-04-30 23:59:59" : "Apr'22",
             "2022-05-31 23:59:59" : "May'22",
             "2022-06-30 23:59:59" : "Jun'22",
             "2022-07-31 23:59:59" : "Jul'22", }

print(len(months_timestamp))

# for date in months_timestamp:
#     print(datetime.strptime(date, '%Y-%m-%d %H:%M:%S').timestamp())

In [ ]:
### Temporal analysis  of RQ1 fine-grained
sub_categories = ['Media/TV',
                  'Home Automation',
                  'Voice Assistant',
                  'Surveillance',]
                  # 'Game Console',]


devices_df = clean_flow_party_label.copy(deep=True)

aggregated_data = []


for category in sub_categories:
    for i in range(2,len(months_timestamp),3):
        # print(months_timestamp[i-1], months_timestamp[i])
        min_ts = datetime.strptime(months_timestamp[i-2], '%Y-%m-%d %H:%M:%S').timestamp()
        max_ts = datetime.strptime(months_timestamp[i+1], '%Y-%m-%d %H:%M:%S').timestamp()
    
        df = devices_df[(devices_df['generic_category']==category) &
                        (devices_df['min_ts']>min_ts) & (devices_df['max_ts']<=max_ts)]


        first = df[df['party_labels']==1].groupby('device_id')['domain'].nunique().reset_index().rename(columns={"domain": "1"})
        support = df[df['party_labels']==2].groupby('device_id')['domain'].nunique().reset_index().rename(columns={"domain": "2"})
        Third = df[df['party_labels']==3].groupby('device_id')['domain'].nunique().reset_index().rename(columns={"domain": "3"})

        devices = df[['device_id']].drop_duplicates()
        devices = devices.merge(first,how ='left').merge(support,how ='left').merge(Third,how='left').fillna(0)

        first_party_mean, support_party_mean, third_party_mean = devices[['1','2','3']].mean()
        
        first_party_up_stream = df[df['party_labels']==1]['average_out_byte_per_sec'].mean()
        support_party_up_stream = df[df['party_labels']==2]['average_out_byte_per_sec'].mean()
        third_party_up_stream = df[df['party_labels']==3]['average_out_byte_per_sec'].mean()
        

        data = {'Category': category,
                'timestamp': max_ts,
                'month': month_map[months_timestamp[i]],
                'first_party_mean': first_party_mean,
                'support_party_mean': support_party_mean,
                'third_party_mean': third_party_mean,
                'first_party_up_stream': first_party_up_stream,
                'support_party_up_stream': support_party_up_stream, 
                'third_party_up_stream': third_party_up_stream
        }
        aggregated_data.append(data)


# Create a DataFrame from the list of aggregated data
rq_1_df = pd.DataFrame(aggregated_data)

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('QtAgg')  # Choose the appropriate backend
plt.style.use('default')  # Set the default style

# Convert 'Timestamp' column to datetime format
# df['Timestamp'] = pd.to_datetime(df['Timestamp'])
# rq_1_df['month-year'] = rq_1_df['timestamp'].dt.strftime('%b-%Y')
# Create a figure and axis for the plot
# fig, ax = plt.subplots(figsize=(12, 6))
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(14, 2.5), sharex=False)
# plt.yscale('log')
# Iterate through unique categories
markers = ['*', 'd', 'o', 'v']
i=0
for category in rq_1_df['Category'].unique():
    # Filter the DataFrame for the current category
    category_data = rq_1_df[rq_1_df['Category'] == category]
    # Plot the values for the current category
    axes[0].plot(category_data['month'], category_data['first_party_mean'], marker=markers[i], label=category)
    axes[1].plot(category_data['month'], category_data['support_party_mean'], marker=markers[i],label=category)
    axes[2].plot(category_data['month'], category_data['third_party_mean'], marker=markers[i], label=category)
    i += 1

# Set labels and title
# axes[0].set_xlabel('Timestamp')
axes[0].set_ylabel('# of Endpoints')
axes[0].set_title('First party endpoints')
axes[1].set_title('Support party endpoints')
axes[2].set_title('Third party endpoints')
# axes[3].legend()
axes[2].legend(loc='center left', bbox_to_anchor=(0, 0.65), prop={'size': 9}, framealpha=0.4)


# Rotate category names vertically
for ax in axes:
    ax.grid(True, axis='y')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='center')
# 


# todo Display and save the plot
plt.tight_layout()
# plt.savefig("../../Statistical Data/RQ4/Temporal-RQ-1-a-marked.pdf", format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('QtAgg')  # Choose the appropriate backend
plt.style.use('default')  # Set the default style


fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(14, 2.5), sharex=False)
plt.yscale('log')
markers = ['*', 'd', 'o', 'v']
i=0
# Iterate through unique categories
for category in rq_1_df['Category'].unique():
    # Filter the DataFrame for the current category
    category_data = rq_1_df[rq_1_df['Category'] == category]
    # Plot the values for the current category
    axes[0].plot(category_data['month'], category_data['first_party_up_stream'], marker=markers[i],  label=category)
    axes[1].plot(category_data['month'], category_data['support_party_up_stream'],  marker=markers[i], label=category)
    axes[2].plot(category_data['month'], category_data['third_party_up_stream'], marker=markers[i],  label=category)
    i += 1

axes[0].set_yscale('log')
axes[1].set_yscale('log')
axes[2].set_yscale('log')

axes[0].set_ylabel('Up-stream Data')

# Rotate category names vertically
for ax in axes:
    ax.grid(True, axis='y')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='center')
# 


# todo Display and save  the plot
plt.tight_layout()
# plt.savefig("../../Statistical Data/RQ4/Temporal-RQ-1-b-marked.pdf", format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
## Temporal analysis of RQ2

devices_df = clean_flow_party_label.copy(deep=True)

aggregated_data = []

locations = ['AMERICAS', 'EA', 'APA']

for category in sub_categories:
    for i in range(2,len(months_timestamp),3):
        # print(months_timestamp[i-1], months_timestamp[i])
        min_ts = datetime.strptime(months_timestamp[i-2], '%Y-%m-%d %H:%M:%S').timestamp()
        max_ts = datetime.strptime(months_timestamp[i+1], '%Y-%m-%d %H:%M:%S').timestamp()

        df = devices_df[(devices_df['generic_category']==category) &
                        (devices_df['min_ts']>min_ts) & (devices_df['max_ts']<=max_ts)]

    
        for loc in locations:
            first = df[(df['user_loc']==loc) & (df['party_labels']==1)].groupby('device_id')['domain'].nunique().reset_index().rename(columns={"domain": "1"})
            support = df[(df['user_loc']==loc) & (df['party_labels']==2)].groupby('device_id')['domain'].nunique().reset_index().rename(columns={"domain": "2"})
            Third = df[(df['user_loc']==loc) & (df['party_labels']==3)].groupby('device_id')['domain'].nunique().reset_index().rename(columns={"domain": "3"})
    
            devices = df[(df['user_loc']==loc)][['device_id']].drop_duplicates()
            devices = devices.merge(first,how ='left').merge(support,how ='left').merge(Third,how='left').fillna(0)
    
            first_party_mean, support_party_mean, third_party_mean = devices[['1','2','3']].mean()
            first_party_up_stream = df[df['party_labels']==1]['average_out_byte_per_sec'].mean()
            support_party_up_stream = df[df['party_labels']==2]['average_out_byte_per_sec'].mean()
            third_party_up_stream = df[df['party_labels']==3]['average_out_byte_per_sec'].mean()


            data = {'Category': category,
                    'timestamp': max_ts,
                    'month': month_map[months_timestamp[i]],
                    'location': loc,
                    'first_party_mean': first_party_mean,
                    'support_party_mean': support_party_mean,
                    'third_party_mean': third_party_mean,
                    'first_party_up_stream': first_party_up_stream,
                    'support_party_up_stream': support_party_up_stream,
                    'third_party_up_stream': third_party_up_stream
                    }
            
            aggregated_data.append(data)


# Create a DataFrame from the list of aggregated data
rq_2_df = pd.DataFrame(aggregated_data)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Get unique categories and subcategories
# categories = rq_2_df['Category'].unique()
categories =['Media/TV', 'Home Automation', 'Voice Assistant']
# subcategories = rq_2_df['location'].unique()
subcategories = ['AMERICAS', 'EA', 'APA']

# Create subplots for each category
fig, axes = plt.subplots(len(categories), 3, figsize=(12,2.5*len(categories)), sharex=False)

for i, category in enumerate(categories):
    category_data = rq_2_df[rq_2_df['Category'] == category]

    # Create a separate subplot for each category
    ax = axes[i, 0]

    # Plot time series data for subcategories within the category
    for subcategory in subcategories:
        subset = category_data[category_data['location'] == subcategory]
        ax.plot(subset['month'], subset['first_party_mean'], label=subcategory)

        # Set labels and title for the subplot
        # ax.set_xlabel('Time')
        ax.set_ylabel('# of First Party Domain')
        ax.set_title(f'Category: {category}')
        ax.grid(True)

    # Create a separate subplot for each category
    ax = axes[i, 1]

    # Plot time series data for subcategories within the category
    for subcategory in subcategories:
        subset = category_data[category_data['location'] == subcategory]
        ax.plot(subset['month'], subset['support_party_mean'], label=subcategory)
        ax.set_ylabel('# of Support Party Domain')
        ax.set_title(f'Category: {category}')
        ax.grid(True)

    # Create a separate subplot for each category
    ax = axes[i, 2]

    # Plot time series data for subcategories within the category
    for subcategory in subcategories:
        subset = category_data[category_data['location'] == subcategory]
        ax.plot(subset['month'], subset['third_party_mean'], label=subcategory)
        ax.set_ylabel('# of Third Party Domain')
        ax.set_title(f'Category: {category}')
        ax.legend()
        ax.grid(True)

for ax in axes[0]:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='center')
for ax in axes[1]:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='center')
for ax in axes[2]:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='center')

# Adjust layout
plt.tight_layout()
plt.savefig("../../Statistical Data/RQ4/Temporal-RQ-2.pdf", format="pdf", bbox_inches="tight")

# Display the subplots
plt.show()

In [ ]:
## Temporal analysis of RQ3
location = ['AMERICAS', 'EA', 'APA']
def flow_over_boarder_updated(dataframe):
    for user_loc in location:
        user_df = dataframe[dataframe['user_loc']==user_loc]
        total_out_byte = user_df[user_df['domain_loc']=='AMERICAS']['total_out_byte'].sum() + user_df[user_df['domain_loc']=='EA']['total_out_byte'].sum() + user_df[user_df['domain_loc']=='APA']['total_out_byte'].sum()

        print('{', end='')
        for domain_loc in location:
            out_byte = user_df[user_df['domain_loc']==domain_loc]['total_out_byte'].sum()
            flow_percentage = out_byte*100/total_out_byte

            if domain_loc != 'APA': print('%.1f' % flow_percentage, end=', ')
            else: print('%.1f' % flow_percentage, end='},\n')
            
            

for category in unique_categories:
    df = devices_df[(devices_df['generic_category']==category) & (devices_df['max_ts']<=threshold)]
    flow_over_boarder_updated(df)
    print(category, '< 2021 \n\n')
    print()

for category in unique_categories:
    df = devices_df[(devices_df['generic_category']==category) & (devices_df['max_ts']>threshold)]
    flow_over_boarder_updated(df)
    print(category, '>= 2021 \n\n')
    print()
    

In [ ]:
### Temporal RQ1 (appendix)
# finding top support parties

from cleanco import basename

support_flows = devices_df[devices_df['party_labels']==2]
support_parties = support_flows['domain'].drop_duplicates().reset_index(drop=True)

# load combined information
file_combined_domain2org = '../../Endpoint Mapping Data/Domain Data/First Party Mapping/domain2org_all_possible_sources.csv'
domain2org = pd.read_csv(file_combined_domain2org).rename(columns={'remote_hostname':'domain'})
# merge 
support_parties = pd.merge(support_parties,
                           domain2org,
                           on=['domain'],
                           how='left'
                           )

# find support organization
support_parties['organization'] = support_parties.apply(lambda row: row.netify_org if row.netify_org==row.netify_org
else basename(basename(row.python_whois_org).capitalize() if (row.python_whois_org==row.python_whois_org) & (len(str(row.python_whois_org).strip())>0)
              else basename(basename(row.bash_whois_org).capitalize() if (row.bash_whois_org==row.bash_whois_org) & (len(str(row.bash_whois_org).strip())>0)
                            else (row.xclusive_org.capitalize() if row.xclusive_org==row.xclusive_org
                                  else row.domain))), axis=1)

support_parties = support_parties[['domain', 'organization']]

# add support organization in the flow table
support_flows = pd.merge(support_flows, support_parties, on='domain', how='left')

# get support organization and count of supported devices 
support_organization = support_flows.groupby(['organization'])[['device_id']].nunique().sort_values(by='device_id', ascending=False).reset_index().rename(columns={'device_id':'device_count'}).set_index('organization')

In [ ]:
## lIST OF VENDORS WITH MULTIPLE SUPPORT PARTIES

vendor_name_device_count = support_flows.groupby(['super_vendor'])[['device_id']].nunique().sort_values(by='device_id', ascending=False)

for vendor in vendor_name_device_count.index:
    device_count = vendor_name_device_count['device_id'].loc[vendor]

    ## skip if device count is less than 3
    if device_count < 10:
        continue

    print(vendor.ljust(15), end=' & \t')

    support_device_count = support_flows[(support_flows['super_vendor']==vendor)  & (support_flows['max_ts']<=threshold)].groupby(['organization'])[['device_id']].nunique().sort_values(by='device_id', ascending=False)
    
    for support in support_device_count.index:
        if support_device_count['device_id'].loc[support] < 3:
            continue
        print('{0}({1}), '.format(support, support_device_count['device_id'].loc[support]), end='')

    print('\\\\')
    print()

    print("", end=' & ')

    support_device_count = support_flows[(support_flows['super_vendor']==vendor)  & (support_flows['max_ts']>threshold)].groupby(['organization'])[['device_id']].nunique().sort_values(by='device_id', ascending=False)
    for support in support_device_count.index:
        if support_device_count['device_id'].loc[support] < 3:
            continue
        print('{0}({1}), '.format(support, support_device_count['device_id'].loc[support]), end='')

    print('\\\\[2pt]')
    print()


In [ ]:
## lIST TOP SUPPORT PARTIES

for org in support_organization.index:
    device_count = support_organization['device_count'].loc[org]

    if device_count < 10:
        continue
    print(org.ljust(15), end=' & \t')

    vendor_device_count = support_flows[(support_flows['organization']==org)  &
                                        (support_flows['max_ts']<=threshold)].groupby(['super_vendor'])[['device_id']].nunique().sort_values(by='device_id', ascending=False)
    for vendor in vendor_device_count.index:
        if vendor_device_count['device_id'].loc[vendor] < 3:
            continue
        print('{0}({1}), '.format(vendor, vendor_device_count['device_id'].loc[vendor]), end='')

    print('\\\\')
    print()

    print("", end=' & ')
    
    vendor_device_count = support_flows[(support_flows['organization']==org)  &
                                        (support_flows['max_ts']>threshold)].groupby(['super_vendor'])[['device_id']].nunique().sort_values(by='device_id', ascending=False)
    
    for vendor in vendor_device_count.index:
        if vendor_device_count['device_id'].loc[vendor] < 3:
            continue
        print('{0}({1}), '.format(vendor, vendor_device_count['device_id'].loc[vendor]), end='')

    print('\\\\[3pt]')
    print()
